In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale
import os

In [36]:
file = "epsilon-3"

### Load in `f` data

In [37]:
df = pd.read_csv(file + "-f.csv", sep=",")
df.T

,0,1,2,3,4,5,6,7,8,9,...,63937,63938,63939,63940,63941,63942,63943,63944,63945,63946
current_player,1.000000,0.000000,1.000000,0.000000,1.000000,0.0,1.000000,0.000000,1.000000,0.0,...,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,1.000000
is_current_player_hand_winner,0.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,0.000000,1.0,...,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,1.000000
is_current_player_game_winner,0.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,0.000000,1.0,...,1.000000,0.00000,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,0.000000,1.000000
current_player_end_hand_score_advantage,-17.000000,17.000000,-17.000000,17.000000,-17.000000,17.0,-17.000000,17.000000,-17.000000,17.0,...,10.000000,-10.00000,10.000000,-10.000000,10.000000,-10.000000,10.00000,26.000000,-26.000000,26.000000
current_player_score,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,68.000000,0.00000,68.000000,0.000000,68.000000,0.000000,68.00000,78.000000,0.000000,78.000000
opponent_score,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,68.00000,0.000000,68.000000,0.000000,68.000000,0.00000,0.000000,78.000000,0.000000
current_player_deadwood,65.000000,44.000000,58.000000,44.000000,58.000000,43.0,58.000000,41.000000,55.000000,41.0,...,18.000000,26.00000,18.000000,26.000000,12.000000,18.000000,8.00000,25.000000,51.000000,10.000000
current_player_num_hit_cards,5.000000,3.000000,7.000000,3.000000,7.000000,5.0,7.000000,6.000000,7.000000,6.0,...,3.000000,5.00000,3.000000,5.000000,3.000000,2.000000,5.00000,8.000000,7.000000,2.000000
num_melds,0.000000,1.000000,0.000000,1.000000,0.000000,1.0,0.000000,1.000000,0.000000,1.0,...,2.000000,1.00000,2.000000,1.000000,2.000000,1.000000,2.00000,1.000000,1.000000,2.000000
point_sum_melds,0.000000,6.000000,0.000000,6.000000,0.000000,6.0,0.000000,6.000000,0.000000,6.0,...,56.000000,9.00000,56.000000,9.000000,56.000000,10.000000,56.00000,30.000000,29.000000,36.000000


In [54]:
f = [
#      'current_player',
#      'is_current_player_hand_winner',
#      'is_current_player_game_winner',
     'current_player_end_hand_score_advantage',
    
     'current_player_score',
     'opponent_score',
     'current_player_deadwood',
     'current_player_num_hit_cards',
     'num_melds',
#      'point_sum_melds',
     'num_combos',
#      'point_sum_combos',
#      'num_knock_cache',
#      'point_sum_knock_cache',
#      'num_load_cards',
#      'point_sum_load_cards',
     'turns_taken',
#      'num_nearby_opponent_cards',
#      'discard_danger'
]


x = f[1:]
y = f[0]

print("x =", x)
print("y =", y)

x = ['current_player_score', 'opponent_score', 'current_player_deadwood', 'current_player_num_hit_cards', 'num_melds', 'num_combos', 'turns_taken']
y = current_player_end_hand_score_advantage


### Load in `m` data

In [55]:
card_matrices = []

with open(file + '-m.csv') as f:
    for line in f.readlines():
        card_matrices.append(eval(line))

card_matrices = np.array(card_matrices)
card_matrices

array([[[0.        , 0.26907994, 0.        , ..., 0.        ,
         0.26907994, 0.26907994],
        [0.26907994, 0.26907994, 0.26907994, ..., 0.26907994,
         0.26907994, 0.26907994],
        [0.26907994, 0.26907994, 0.26907994, ..., 0.26907994,
         0.26907994, 0.        ],
        ...,
        [0.26907994, 0.26907994, 0.26907994, ..., 0.26907994,
         0.26907994, 0.        ],
        [0.        , 0.26907994, 0.        , ..., 0.        ,
         0.26907994, 0.26907994],
        [0.26907994, 0.26907994, 0.26907994, ..., 0.26907994,
         0.26907994, 0.26907994]],

       [[0.26997829, 0.26986806, 0.27000489, ..., 0.        ,
         0.26110739, 0.        ],
        [0.26869792, 0.        , 0.26820064, ..., 0.24311868,
         0.29109001, 0.3146228 ],
        [0.26780297, 0.        , 0.        , ..., 0.21708868,
         0.25383154, 0.28658311],
        ...,
        [0.26780297, 0.        , 0.        , ..., 0.21708868,
         0.25383154, 0.28658311],
        [0.2

### split train and test data

In [56]:
trainFeaturesX, testFeaturesX, trainMatricesX, testMatricesX = train_test_split(df[x], card_matrices, test_size=0.25, random_state=42)
trainY, testY = train_test_split(df[[y]], test_size=0.25, random_state=42)

display(trainFeaturesX.T)
display(trainY.T)

,39308,30670,5046,35363,49543,7505,16796,3125,40012,30200,...,6265,54886,11284,44732,54343,62570,38158,860,15795,56422
current_player_score,16.0,67.0,0.0,38.0,0.0,21.0,21.0,31.0,40.0,72.0,...,23.0,59.0,87.0,34.0,35.0,0.0,48.0,36.0,70.0,27.0
opponent_score,91.0,54.0,28.0,19.0,0.0,24.0,36.0,34.0,24.0,91.0,...,20.0,49.0,78.0,39.0,58.0,0.0,94.0,54.0,91.0,10.0
current_player_deadwood,37.0,42.0,20.0,28.0,8.0,6.0,54.0,28.0,27.0,25.0,...,14.0,63.0,36.0,33.0,10.0,35.0,22.0,10.0,38.0,41.0
current_player_num_hit_cards,6.0,5.0,3.0,4.0,3.0,7.0,7.0,6.0,4.0,7.0,...,9.0,4.0,9.0,5.0,2.0,6.0,6.0,4.0,2.0,9.0
num_melds,1.0,1.0,2.0,1.0,2.0,2.0,0.0,1.0,1.0,1.0,...,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0
num_combos,2.0,2.0,1.0,2.0,1.0,3.0,3.0,3.0,2.0,4.0,...,3.0,2.0,4.0,2.0,0.0,3.0,2.0,1.0,1.0,4.0
turns_taken,6.0,1.0,10.0,9.0,11.0,8.0,6.0,15.0,11.0,9.0,...,13.0,3.0,1.0,1.0,5.0,6.0,10.0,10.0,1.0,1.0


,39308,30670,5046,35363,49543,7505,16796,3125,40012,30200,...,6265,54886,11284,44732,54343,62570,38158,860,15795,56422
current_player_end_hand_score_advantage,5.0,-15.0,5.0,-19.0,20.0,26.0,-34.0,-18.0,5.0,-4.0,...,-10.0,-19.0,10.0,6.0,16.0,-3.0,-13.0,36.0,39.0,55.0


### create the MLP model

In [57]:
mlp = Sequential()
mlp.add(Dense(8, input_dim=trainFeaturesX.shape[1], activation="relu"))
mlp.add(Dense(4, activation="relu"))
if False:
    mlp.add(Dense(1, activation = "linear"))

### Create CNN model

In [58]:
height = len(card_matrices[0])
width = len(card_matrices[0][0])
depth = 1
filters = (16, 32, 64)

inputShape = (height, width, depth)
chanDim = -1

inputs = Input(shape=inputShape)

for (i, f) in enumerate(filters):
    # if this is the first CONV layer then set the input appropriately
    if i == 0:
        x = inputs

    # CONV => RELU => BN => POOL
    x = Conv2D(f, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

# flatten the volume, then FC => RELU => BN => DROPOUT
x = Flatten()(x)
x = Dense(16)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=chanDim)(x)
x = Dropout(0.5)(x)

# match the number of nodes coming out of the MLP
x = Dense(4)(x)
x = Activation("relu")(x)

if False:
    x = Dense(1, activation="linear")(x)

cnn = Model(inputs, x)

### Combine

In [59]:
combinedInput = concatenate([mlp.output, cnn.output])

x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

model = Model(inputs=[mlp.input, cnn.input], outputs=x)

### Compile

In [60]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

### Train

In [61]:
model.fit(
    [trainFeaturesX, trainMatricesX], trainY,
    validation_data=([testFeaturesX, testMatricesX], testY),
    epochs=200, batch_size=25)

Epoch 1/200
1919/1919 [==============================] - 12s 6ms/step - loss: 99.9639 - val_loss: 99.7024
Epoch 2/200
1919/1919 [==============================] - 12s 6ms/step - loss: 99.5130 - val_loss: 100.0122
Epoch 3/200
1919/1919 [==============================] - 12s 6ms/step - loss: 98.9172 - val_loss: 98.6158
Epoch 4/200
1919/1919 [==============================] - 12s 6ms/step - loss: 97.8626 - val_loss: 98.4194
Epoch 5/200
1919/1919 [==============================] - 12s 6ms/step - loss: 97.3021 - val_loss: 97.4541
Epoch 6/200
1919/1919 [==============================] - 12s 6ms/step - loss: 96.6592 - val_loss: 97.3297
Epoch 7/200
1919/1919 [==============================] - 12s 6ms/step - loss: 96.6331 - val_loss: 97.0646
Epoch 8/200
1919/1919 [==============================] - 12s 6ms/step - loss: 96.0675 - val_loss: 97.1854
Epoch 9/200
1919/1919 [==============================] - 12s 6ms/step - loss: 95.3589 - val_loss: 95.6835
Epoch 10/200
1919/1919 [=====================

### Testing

In [62]:
preds = model.predict([testFeaturesX, testMatricesX])
diff = (preds.T[0] - testY.values.T[0])**2

print("average difference =", np.sqrt(np.average(diff)))

average difference = 16.467009511527657


In [63]:
print(model.predict([testFeaturesX, testMatricesX]).T[0][:10])

[ 0.9986059 -5.3221855  4.154753   7.296451   5.4856796  4.132338
  2.663418  -4.7558055  1.6129196  8.858318 ]


In [64]:
print(testY.values.T[0][:10])

[ 37. -18.  24. -11.   3. -12.   5. -13. -28.   4.]


In [49]:
model.save('epsilon.h5')